In [1]:
import json
import keras
import numpy as np

/usr/lib/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
with open('./kvret_dataset_public/kvret_train_public.json') as f:
    dat=json.load(f)

In [3]:
with open('./kvret_dataset_public/kvret_dev_public.json') as f:
    valid_dat=json.load(f)

In [4]:
dat[1].keys()

dict_keys(['dialogue', 'scenario'])

In [5]:
dat[1]['dialogue'][0].keys()

dict_keys(['turn', 'data'])

In [6]:
dat[1]['dialogue'][0]['turn']

'driver'

In [7]:
dat[1]['dialogue'][0]['data']

{'end_dialogue': False, 'utterance': 'Where is the nearest gas station?'}

In [8]:
dat[12]['dialogue'][1]['data']

{'end_dialogue': False,
 'requested': {'distance': True,
  'traffic_info': False,
  'poi_type': True,
  'address': True,
  'poi': False},
 'slots': {'distance': 'nearest', 'poi_type': 'shopping mall'},
 'utterance': "The nearest shopping mall is Stanford Shopping Center. It's located 3 miles away and there is no traffic to reach it"}

In [9]:
dat[10]['scenario']

{'kb': {'items': [{'monday': 'drizzle, low of 80F, high of 90F',
    'tuesday': 'rain, low of 80F, high of 90F',
    'friday': 'blizzard, low of 20F, high of 30F',
    'wednesday': 'snow, low of 40F, high of 60F',
    'thursday': 'cloudy, low of 50F, high of 70F',
    'sunday': 'foggy, low of 90F, high of 100F',
    'location': 'inglewood',
    'saturday': 'blizzard, low of 30F, high of 40F',
    'today': 'monday'},
   {'monday': 'cloudy, low of 60F, high of 80F',
    'tuesday': 'humid, low of 80F, high of 90F',
    'friday': 'blizzard, low of 40F, high of 50F',
    'wednesday': 'drizzle, low of 90F, high of 100F',
    'thursday': 'clear skies, low of 60F, high of 70F',
    'sunday': 'humid, low of 80F, high of 100F',
    'location': 'san mateo',
    'saturday': 'overcast, low of 90F, high of 100F',
    'today': 'monday'},
   {'monday': 'snow, low of 20F, high of 30F',
    'tuesday': 'dry, low of 70F, high of 80F',
    'friday': 'windy, low of 90F, high of 100F',
    'wednesday': 'fros

# Analyzing first

In [10]:
from collections import defaultdict
dic = defaultdict(str)
for d in dat:
    for _d in d['dialogue']:
        dic.update(_d['data'].get('slots',{}))

In [11]:
belief_state_names = list(dic.keys())

In [12]:
dic

defaultdict(str,
            {'distance': 'best possible route',
             'poi_type': 'coffee ot tea place',
             'traffic_info': 'avoid all heavy traffic',
             'poi': "chef chu's",
             'date': '1st',
             'party': 'Ana',
             'event': 'Swimming activity',
             'time': '2pm',
             'location': 'Seattle',
             'weather_attribute': '7 day forecast',
             'room': 'conference room 100',
             'address': '271 Springer Street',
             'agenda': 'medicine'})

In [13]:
from collections import Counter
dic = set([])
for d in dat:
    for _d in d['dialogue']:
        slots = _d['data'].get('slots')
        if not slots: continue
        dic.update([slots.get('distance')])

In [14]:
from collections import defaultdict
dic = defaultdict(str)
for d in dat:
    for _d in d['dialogue']:
        dic.update(_d.get('data',{}))

In [15]:
requested_slots = list(dic['requested'].keys())

In [16]:
#Kb

In [17]:
d['scenario'].keys()

dict_keys(['kb', 'task', 'uuid'])

In [18]:
s=set([])
for d in dat:
    try:
        keys=d['scenario']['kb']['column_names']
    except:
        continue
    s.update(keys)
print(s)

{'date', 'party', 'address', 'event', 'distance', 'monday', 'tuesday', 'wednesday', 'location', 'traffic_info', 'poi_type', 'time', 'thursday', 'room', 'friday', 'saturday', 'sunday', 'today', 'poi', 'agenda'}


In [19]:
s=set([])
for d in dat:
    try:
        keys=[t['traffic_info'] for t in d['scenario']['kb']['items'] ]
    except:
        continue
    s.update(keys)
print(s)

{'heavy traffic', 'road block nearby', 'no traffic', 'moderate traffic', 'car collision nearby'}


In [20]:
s=set([])
for d in dat:
    try:
        keys=[d['scenario']['kb']['kb_title']]
    except:
        continue
    s.update(keys)
print(s)

{'location information', 'weekly forecast', 'calendar'}


In [21]:
s=set([])
for d in dat:
    try:
        keys=[d['scenario']['task']['intent']]
    except:
        print(d)
        continue
    s.update(keys)
print(s)

{'schedule', 'weather', 'navigate'}


In [22]:
s=defaultdict(set)
for d in dat:
    try:
        s[d['scenario']['kb']['kb_title']].update([d['scenario']['task']['intent']])
    except:
        print(d)
        continue
print(s)

defaultdict(<class 'set'>, {'location information': {'navigate'}, 'calendar': {'schedule'}, 'weekly forecast': {'weather'}})


In [23]:
s=defaultdict(set)
for d in dat:
    try:
        s[d['scenario']['task']['intent']].update(d['scenario']['kb']['column_names'])
    except Exception as e:
        print(e)
        continue
print(s)

defaultdict(<class 'set'>, {'navigate': {'address', 'distance', 'traffic_info', 'poi_type', 'poi'}, 'schedule': {'date', 'party', 'event', 'time', 'room', 'agenda'}, 'weather': {'tuesday', 'wednesday', 'monday', 'location', 'saturday', 'thursday', 'friday', 'sunday', 'today'}})


In [24]:
s=defaultdict(set)
for d in dat:
    try:
        if d['scenario']['task']['intent'] == 'weather':
            print(d)
            print('\n\n\n\n')
            break
    except Exception as e:
        print(e)
        continue
print(s)

{'dialogue': [{'turn': 'driver', 'data': {'end_dialogue': False, 'utterance': 'check forecast for new york'}}, {'turn': 'assistant', 'data': {'end_dialogue': False, 'requested': {'weather_attribute': True, 'date': False, 'location': True}, 'slots': {'location': ' new york'}, 'utterance': 'The weather this week will be hail and rain on Thursday and Sunday, with the lowest temperature being 20F and the highest 90F, Monday will be the only hot day. '}}, {'turn': 'driver', 'data': {'end_dialogue': False, 'utterance': 'Will it be overcast on friday?'}}, {'turn': 'assistant', 'data': {'end_dialogue': False, 'requested': {'weather_attribute': True, 'date': True, 'location': False}, 'slots': {'weather_attribute': 'overcast', 'date': 'Friday'}, 'utterance': 'On Friday it will be cloudy on New York'}}, {'turn': 'driver', 'data': {'end_dialogue': False, 'utterance': 'Thank you!'}}, {'turn': 'assistant', 'data': {'end_dialogue': True, 'requested': {'weather_attribute': False, 'date': False, 'locat

# KB results with beilef states

In [25]:
belief_state_names

['distance',
 'poi_type',
 'traffic_info',
 'poi',
 'date',
 'party',
 'event',
 'time',
 'location',
 'weather_attribute',
 'room',
 'address',
 'agenda']

In [26]:
defaultdict(lambda: 'str')['hi']

'str'

In [27]:
from difflib import SequenceMatcher

def sim(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def close(a,b):
    return sim(a,b)>=0.50

In [28]:
import re
import dateutil

In [29]:
def num_op(s1,op,s2):
    if isinstance(s1, (int, float)):
        num1 = s1
    else:
        m1=re.search(r'([0-9]+)',s1)
        if m1 is None:
            try:
                num1=dateutil.parser.parse(s1)
            except ValueError:
                return False,None
        num1=int(m1.group())
           
    if isinstance(s2,(int,float)):
        num2 = s2
    else:
        m2=re.search(r'([0-9]+)',s2)
        if m2 is None:
            try:
                num2=dateutil.parser.parse(s2)
            except ValueError:
                return False ,None     
        num2=int(m2.group())
    try:
        if op=='equal to':
            return num1==num2,num1
        if op=='greater than':
            return num1>num2,num1
        if op=='less than':
            return num1<num2,num1
    except TypeError:
        return False,num1

In [30]:
def kb_results(kb_data,kb_intent,columns,pred_kb_intent,belief_state,operation):
    """This function gets the kb_data, column names and intent for which the kb is received. 
    If intent identified by the bot is nto same as kb_intent no results will be returned.
    
    intents types: {'schedule', 'weather', 'navigate'}
    
    operation should have same keys as belief state with following possible values:
    str, =, >, <, minimum, maximum  indexed from 0 to 5
    Use None for all the values not numerical. If not None, operation would be performend
    """
    #defaultdict(<class 'set'>, {'navigate': {'poi', 'distance', 'poi_type', 'traffic_info', 'address'},, 'weather': {'thursday', 'sunday', 'today', 'friday', 'wednesday', 'tuesday', 'saturday', 'location', 'monday'}})
    if pred_kb_intent!=kb_intent:
        return [],[]
    if kb_data is None:
        return [],[]
    results = np.ones(len(kb_data))
    confidence = np.ones(len(kb_data))
    # column names possiblity: {'room', 'party', 'event', 'agenda', 'date', 'time'}  
    # Note that date and time are immutable and non-comparable in current dialog, so they are treated as strings
    col_types = defaultdict(lambda: 'str')
    if any(k not in columns for k in belief_state.keys()):
        return [],[]
    for k in belief_state.keys():
        min_idx = None
        min_val = float('Inf')
        max_idx = None
        max_val = -float('Inf')
        for i,items in enumerate(kb_data):
            if results[i] == 0: continue
            if col_types[k]=='str':
                if not (belief_state.get(k) and items.get(k) and operation.get(k) is not None):
                    results[i]=0
                    confidence[i]=0
                else:
                    results[i]=0
                    if operation[k]==0 and close(belief_state[k],items[k]):
                        # Doing string comparison
                        results[i]=1
                        confidence[i] = confidence[i]*sim(belief_state.get(k),items.get(k))
                    elif operation[k]==1:
                        #Doing equal comparison extracting the first number
                        if num_op(belief_state[k],'equal to',items[k])[0]:
                            results[i]=1
                    elif operation[k]==2:
                        #Doing greater than comparison extracting the first number
                        if num_op(belief_state[k],'less than',items[k])[0]:
                            results[i]=1
                    elif operation[k]==3:
                        #Doing less than comparison extracting the first number
                        if num_op(belief_state[k],'greater than',items[k])[0]:
                            results[i]=1
                    elif operation[k]==4:
                        #Doing mimum comparison extracting the first number
                        res,val = num_op(items[k],'less than',(min_val))
                        if res:
                            results[i]=1
                            if min_idx is not None:
                                results[min_idx] = 0
                                confidence[min_idx] = 0
                            min_val = val
                            min_idx = i
                    elif operation[k]==5:
                        #Doing maximum comparison extracting the first number
                        res,val = num_op(items[k],'greater than',(max_val))
                        if res:
                            results[i]=1
                            if max_idx is not None:
                                results[max_idx] = 0
                                confidence[max_idx] = 0
                            max_val = val
                            max_idx = i
    
           
    return np.array(results),np.array(confidence)

In [31]:
kb_data = dat[22]['scenario']['kb']['items']
kb_intent = dat[22]['scenario']['task']['intent']
columns = dat[22]['scenario']['kb']['column_names']

In [32]:
pred_kb_intent = kb_intent
belief_state = {'traffic_info':'no traffic'}
operation = {'traffic_info':0}

In [33]:
kb_results(kb_data,kb_intent,columns,pred_kb_intent,belief_state,operation)

(array([1., 0., 1., 0., 1., 1., 1., 0.]),
 array([0.69230769, 1.        , 1.        , 1.        , 0.69230769,
        1.        , 0.69565217, 1.        ]))

# Preprocessing

In [34]:
from nltk import word_tokenize
from tqdm import tqdm
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

In [35]:
def tokenize(t):
    return word_tokenize(t)

In [36]:
# collection of kb documents
doc_kb=[]
for d in tqdm(dat):
    try:
        for item in d['scenario']['kb']['items']:
            [doc_kb.append(t)  for t in item.values()]
    except TypeError:
        continue

100%|██████████| 2425/2425 [00:00<00:00, 105878.18it/s]


In [37]:
doc_colnames=[]
for d in tqdm(dat):
    try:
        item = d['scenario']['kb']['column_names']
        [doc_colnames.append(t)  for t in item]
    except TypeError:
        continue
    

100%|██████████| 2425/2425 [00:00<00:00, 302075.59it/s]


In [38]:
all_columns=list(set(doc_colnames))

In [39]:
# Vocabulary for the databases
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(filters="")

In [40]:
tokenizer.fit_on_texts(doc_kb)

In [41]:
tokenizer.fit_on_texts(["<SOS>","<EOS>"])

# Training the db model first

In [42]:
all_columns_wi={a:i for i,a in enumerate(all_columns)}

In [43]:
assert all_columns
assert tokenizer

In [44]:
MAX_QUERIES = 1
NUM_COL = len(all_columns)
DB_VOCAB_LEN = len(tokenizer.word_index)+1
THRESHOLD = 0.5
MAX_DB_RESULTS = 5
MAX_ENTITY_LENGTH = 10
OPERATOR_LEN = 6
NUM_INTENTS = 3
EMBEDDING_SIZE=50

In [45]:
result = np.zeros((5,6))

In [46]:
# Converting rules based db to desired output first
def results_to_vector(bs_output,pred_intent,operation,kb_data,kb_intent):
    assert bs_output.shape == (NUM_COL,MAX_ENTITY_LENGTH,DB_VOCAB_LEN)
    assert operation.shape == (NUM_COL,OPERATOR_LEN)
    pred_intent = np.argmax(pred_intent) if max(pred_intent)>THRESHOLD else None
    kb_intent = np.argmax(kb_intent)
    output=np.zeros((MAX_DB_RESULTS,NUM_COL,MAX_ENTITY_LENGTH,DB_VOCAB_LEN))
    if intent is None:
        return output
    q=bs_output
    op = operation
    op_conf =  np.max(op,axis=-1)
    op_classes = np.argmax(op,axis=-1) 
    op_classes = [_q if _q_conf>THRESHOLD else None for _q,_q_conf in zip(op_classes,op_conf)]

    q_ents = np.argmax(q,axis=-1)
    q_confs = np.max(q,axis=-1)
    q_mask = np.array(q_confs>THRESHOLD,dtype='float32')
    q_ents = q_mask*q_ents
    q_words = [" ".join([tokenizer.index_word[_q] for _q in __q if _q!=0]) for __q in q_ents]
    # Now that q_words and op_classes are known
    bs={}
    operations = {}
    for j,ent in enumerate(q_words):
        if ent is None or ent=="": continue
        bs[all_columns[j]]=ent
        operations[all_columns[j]] = op_classes[j]
    result,confidence = kb_results(kb_data,kb_intent,columns,pred_intent,bs,operations)
    result=np.array(result)
    confidence=np.array(confidence)
    result = result[np.argsort(confidence)[-1::-1]]
    confidence = confidence[np.argsort(confidence)[-1::-1]]
    final_result=[kb_data[_i] for _i,(c,r) in enumerate(zip(confidence,result)) if c>=THRESHOLD and r==1]
    confidence=[confidence[_i] for _i,(c,r) in enumerate(zip(confidence,result)) if c>=THRESHOLD and r==1]
    kb_result = np.zeros((MAX_DB_RESULTS,NUM_COL,MAX_ENTITY_LENGTH,DB_VOCAB_LEN))
    for j,r in enumerate(final_result):
        if j==MAX_DB_RESULTS: break
        for k,v in r.items():
            kb_result[j,all_columns_wi[k]] = to_categorical(pad_sequences(tokenizer.texts_to_sequences([v]),maxlen=MAX_ENTITY_LENGTH)\
                                                             ,num_classes=DB_VOCAB_LEN)*confidence[j]

    output = kb_result
    return output

In [47]:
bs_output=np.zeros((NUM_COL,MAX_ENTITY_LENGTH,DB_VOCAB_LEN))
intent=np.array([1,0,0])
operation = np.zeros((NUM_COL,OPERATOR_LEN))
bs_output[4,0:2]=to_categorical(tokenizer.texts_to_sequences(['no traffic'])[0],num_classes=DB_VOCAB_LEN)
operation[4] = np.array([1,0,0,0,0,0])
kb_data = dat[0]['scenario']['kb']['items']

In [48]:
np.max(results_to_vector(bs_output,intent,operation,kb_data,intent))

0.0

In [49]:
bs_output=np.zeros((NUM_COL,MAX_ENTITY_LENGTH,DB_VOCAB_LEN))
intent=np.array([1,0,0])
operation = np.zeros((NUM_COL,OPERATOR_LEN))
bs_output[19,0]=to_categorical(tokenizer.texts_to_sequences(['dish'])[0],num_classes=DB_VOCAB_LEN)
operation[19,0]=1.0
kb_data = dat[0]['scenario']['kb']['items']

In [50]:
from keras.layers import Input,Dense,LSTM,Embedding,TimeDistributed, RepeatVector, Concatenate,Reshape
from keras.layers import Lambda
from keras.models import Model

In [51]:
import keras.backend as K
import tensorflow as tf

In [52]:
# model for db
bs_input = Input(shape=(MAX_QUERIES,NUM_COL,MAX_ENTITY_LENGTH,DB_VOCAB_LEN))
intent_input = Input(shape=(MAX_QUERIES,NUM_INTENTS,))
operation_input = Input(shape=(MAX_QUERIES,NUM_COL,OPERATOR_LEN))

LSTM_bs_emb = TimeDistributed(TimeDistributed(LSTM(50,return_sequences=False,return_state=False)))(bs_input)
rep_intent_input = TimeDistributed(RepeatVector(NUM_COL))(intent_input)
print(LSTM_bs_emb.shape)
all_steps = Concatenate(axis=-1)([LSTM_bs_emb,operation_input,rep_intent_input])
all_steps = Lambda(lambda x: tf.reshape(x,shape=(-1,MAX_QUERIES,NUM_COL*(50+OPERATOR_LEN+NUM_INTENTS))))(all_steps)
encoder_lstm = Dense(50,activation='relu')(all_steps)
encoder_lstm = TimeDistributed(RepeatVector(MAX_DB_RESULTS))(encoder_lstm)

decoder_lstm1 = TimeDistributed(LSTM(50,return_sequences=True))(encoder_lstm)

decoder_lstm1 = Dense(NUM_COL*50,activation='relu')(decoder_lstm1)
decoder_lstm1 = Lambda(lambda x: tf.reshape(x,shape=(-1,MAX_QUERIES,MAX_DB_RESULTS,NUM_COL,50)))(decoder_lstm1)


decoder_lstm2 = TimeDistributed(Lambda(lambda x: K.tile(K.expand_dims(x,axis=-2),[1,1,1,MAX_ENTITY_LENGTH,1])))(decoder_lstm1)
decoder_lstm3 = TimeDistributed(TimeDistributed(TimeDistributed(LSTM(50,return_sequences=True))))(decoder_lstm2)

out = Dense(DB_VOCAB_LEN,activation='softmax')(decoder_lstm3)
db_model = Model(inputs=[bs_input,intent_input,operation_input],outputs=[out])

(?, 1, 20, 50)


In [53]:
db_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 20, 10, 38 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 3)         0                                            
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 1, 20, 50)    88000       input_1[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1, 20, 6)     0                                            
__________________________________________________________________________________________________
time_distr

In [54]:
db_model.compile(optimizer='adam',loss='categorical_crossentropy')

### Preparing training inputs

In [55]:
def softmax(w, t = 1.0):
    e = np.exp(np.array(w) / t)
    dist = e / np.sum(e)
    return dist

In [56]:
intents = ['schedule', 'weather', 'navigate']
def input_generator(batch_size,data=dat):
    batch_data1=[]
    batch_data2=[]
    batch_data3=[]
    target=[]
    random_dat = [data[i] for i in np.random.permutation(len(data))]
    ij=0
    while True:
        ij+=1
        for d in random_dat:
            kb_intent = d['scenario']['task']['intent']
            kb_col_names = d['scenario']['kb']['column_names']
            kb_data = d['scenario']['kb']['items']
            true_vec_intent = np.zeros(NUM_INTENTS)
            true_vec_intent[intents.index(kb_intent)]=1.0
            pred_intent = softmax(np.random.normal(size=NUM_INTENTS,loc=100,scale=5))
            bs_input = np.zeros((NUM_COL,MAX_ENTITY_LENGTH,DB_VOCAB_LEN))
            operation = np.zeros((NUM_COL,OPERATOR_LEN))
            num_cols_to_have = np.random.randint(NUM_COL)
            num_ents_to_have = [np.random.randint(NUM_COL) for _ in range(num_cols_to_have)]
            for ii in range(num_cols_to_have):
                for j in range(num_ents_to_have[ii]):
                    ix=(np.random.randint(NUM_COL),np.random.randint(MAX_ENTITY_LENGTH))
                    bs_input[ix] = softmax(np.random.normal(size=DB_VOCAB_LEN,loc=100,scale=5))
                operation[np.random.randint(NUM_COL)] = softmax(np.random.normal(size=OPERATOR_LEN,loc=100,scale=5))
            batch_data1.append(np.array([bs_input]))
            batch_data2.append(np.array([operation]))
            batch_data3.append(np.array([pred_intent]))
            target.append(np.array([results_to_vector(bs_input,pred_intent,operation,kb_data,true_vec_intent)]))
            
            if len(batch_data1)==batch_size:
                yield [np.array(batch_data1),np.array(batch_data3),np.array(batch_data2)],np.array(target)
                batch_data1=[]
                batch_data2=[]
                batch_data3=[]
                target=[]

In [57]:
next(input_generator(1,dat))[0][0].shape

(1, 1, 20, 10, 389)

In [58]:
import psutil
def memory_exhausted():
    if memory_percent_available()<=10:
        print("Memory Exhausted")
        exit()
class memCall(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs={}):
        memory_exhausted()
        
def memory_percent_available():
    return psutil.virtual_memory().available/psutil.virtual_memory().total*100


In [59]:
checkpoint=keras.callbacks.ModelCheckpoint('./db_model.h5',save_best_only=True)

In [61]:
batch_size=16
db_model.fit_generator(input_generator(batch_size),validation_data=input_generator(batch_size,valid_dat),steps_per_epoch=1250,epochs=10,validation_steps=650,callbacks=[memCall(),checkpoint])

Epoch 1/10
1250/1250 [==============================] - 651s 521ms/step - loss: 0.0103 - val_loss: 0.0097
Epoch 2/10
1250/1250 [==============================] - 651s 521ms/step - loss: 0.0088 - val_loss: 0.0070
Epoch 3/10
1250/1250 [==============================] - 657s 526ms/step - loss: 0.0070 - val_loss: 0.0062
Epoch 4/10
 151/1250 [==>...........................] - ETA: 7:51 - loss: 0.0062

KeyboardInterrupt: 

In [62]:
db_model.load_weights('./db_model.h5')

In [63]:
kb_data = dat[22]['scenario']['kb']['items']
kb_intent = dat[22]['scenario']['task']['intent']
columns = dat[22]['scenario']['kb']['column_names']

In [64]:
pred_kb_intent = kb_intent
belief_state = {'traffic_info':'no traffic','distance':'6 miles'}
operation = {'traffic_info':0,'distance':5}

In [65]:
kb_results(kb_data,kb_intent,columns,pred_kb_intent,belief_state,operation)

(array([0., 0., 0., 0., 0., 0., 1., 0.]),
 array([0.        , 1.        , 0.        , 1.        , 0.69230769,
        1.        , 0.69565217, 1.        ]))

In [66]:
tokenizer.texts_to_sequences(['no traffic'])

[[39, 19]]

In [67]:
bs_output=np.zeros((NUM_COL,MAX_ENTITY_LENGTH,DB_VOCAB_LEN))
intent=np.array([1,0,0])
operation = np.zeros((NUM_COL,OPERATOR_LEN))
bs_output[4,0:2]=to_categorical(tokenizer.texts_to_sequences(['no traffic'])[0],num_classes=DB_VOCAB_LEN)
operation[4] = np.array([1,0,0,0,0,0])
kb_data = dat[0]['scenario']['kb']['items']

In [68]:
act=results_to_vector(bs_output,intent,operation,kb_data,intent)

In [70]:
pred=db_model.predict([np.array([[bs_output]]),np.array([[intent]]),np.array([[operation]])])

In [71]:
np.argmax(act[0][0],axis=-1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [73]:
np.argmax(pred[0][0][2],axis=-1)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 2, 1, 1, 1, 1, 1],
       [0, 0, 0, 2, 2, 1, 1, 1, 1, 1],
       [0, 0, 0, 2, 2, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 2, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 2, 1, 1, 1, 1, 1],
       [0, 0, 0, 2, 2, 1, 1, 1, 1, 1],
       [0, 0, 0, 2, 2, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

# Training the NLG model first

In [ ]:
389*20*10*5

This model would receive the belief states and give out the sentences in the coeherent form

In [ ]:
inp = Input((10,vocab_len))
encoder = LSTM(50,return_state=True)
out,out_h,out_c=encoder(inp)
out = Dense(vocab_len,activation='softmax')(out)
model_rnn_2=Model(inputs=[inp],outputs=out)

out_3 = frozen_db_model(model_rnn_2.output)
out_35 = RepeatVector(10)(out_3)
decoder = LSTM(50,return_sequences=True,return_state=True)
out_4,_,_= decoder(out_35,initial_state=[out_h,out_c])
out_4 = Dense(vocab_len,activation='softmax')(out_4)
model_4 = Model(inputs=[model_rnn_2.input],outputs=[model_rnn_2.output,out_4])